# Sugges_

One of the strategies to improve user experience is to provide user with hints, or, otherwise, to autocomplete his queries. Let's consider suggest.

Today we will practice generating suggestions using [Trie](https://en.wikipedia.org/wiki/Trie) data structure (prefix tree), see the example below.

Plan of your homework:

1. Build Trie based on real search query data, provided by AOL company;
2. Generate suggestion based on a trie;
3. Measure suggestion speed;

![image](https://www.ritambhara.in/wp-content/uploads/2017/05/Screen-Shot-2017-05-01-at-4.01.38-PM.png)

## 0. Install Trie data structure support

You are free to use any library implementation of Trie, as well as the one we suggest (read the docs before asking any questions!): https://github.com/google/pygtrie

In [45]:
!pip install pygtrie


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Build a trie upon a dataset

### 1.1. [5] Read the dataset

Download the [dataset](https://github.com/IUCVLab/information-retrieval/tree/main/datasets/aol) (we provide only the first part of the original data for simplicity (~3.5 mln queries)).

Explore the data, see readme file. Load the dataset. Pass the assert.

In [46]:
!tar -xf user-ct-test-collection-01.txt.gz

tar: Error opening archive: Unrecognized archive format


In [47]:
import pandas as pd

aol_data = pd.read_csv('user-ct-test-collection-01.txt', sep='\t')
assert aol_data.shape[0] == 3558411, "Dataset size does not match"

In [48]:
from IPython.display import display
aol_data = aol_data.dropna(subset=['Query'])    
aol_data = aol_data.drop(aol_data[aol_data.Query == '-'].index)
display(aol_data)

,AnonID,Query,QueryTime,ItemRank,ClickURL
0,142,rentdirect.com,2006-03-01 07:17:12,NaN,NaN
1,142,www.prescriptionfortime.com,2006-03-12 12:31:06,NaN,NaN
2,142,staple.com,2006-03-17 21:19:29,NaN,NaN
3,142,staple.com,2006-03-17 21:19:45,NaN,NaN
4,142,www.newyorklawyersite.com,2006-03-18 08:02:58,NaN,NaN
...,...,...,...,...,...
3558405,24967786,www.giantfoods.com,2006-05-31 09:42:18,NaN,NaN
3558407,24969251,sp.trafficmarketplace.com,2006-05-31 15:51:23,NaN,NaN
3558408,24969374,orioles tickets,2006-05-31 12:24:51,NaN,NaN
3558409,24969374,orioles tickets,2006-05-31 12:31:57,2.0,http://www.greatseats.com


### 1.2. [10] Build a Trie

We want a suggest function to be **non-sensitive to stop words** because we don't want to upset the users if they confuses/omits prepositions. Consider *"public events in Innopolis"* vs *"public events at Innopolis"* or *"public events Innopolis"* - they all mean the same.

Build a Trie based on the dataset, **storing query statistics such as query _frequency_, urls and ranks in the nodes**. Some queries may have no associated urls, others may have multiple ranked urls. Think of the way to store this information.

Pass the asserts.

In [ ]:
import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

In [56]:
import pygtrie
from functools import reduce

def remove_stops(query: str) -> str:
    return " ".join([x for x in query.split() if x not in stop_words])

def build_trie(data: pd.DataFrame) -> pygtrie.CharTrie:
    trie = pygtrie.CharTrie()
    for _, row in data.iterrows():
        query, rank, url = row['Query'], row["ItemRank"], row["ClickURL"]
        
        key = remove_stops(query.lower())
        
        if not trie.has_key(key):
            trie[key] = {}

        trie[key].setdefault(query, []).append((rank, url))

    return trie

aol_trie = build_trie(aol_data)

In [57]:
# test trie
# print(aol_trie)
bag = []
for key, val in aol_trie.iteritems("sample q"):
    print(key, '~', val)

    # NB: here we assume you store urls in a property of list type. But you can do something different.
    ranks_urls = [rank_url for ranks_urls in val.values() for rank_url in ranks_urls] 
    bag += [item[1] for item in ranks_urls]
    
    assert "sample question" in key, "All examples have `sample question` substring"
    assert key[:len("sample question")] == "sample question", "All examples have `sample question` starting string"

for url in ["http://www.surveyconnect.com", "http://www.custominsight.com",
            "http://jobsearchtech.about.com", "http://www.troy.k12.ny.us",
            "http://www.flinders.edu.au", "http://uscis.gov"]:
    assert url in bag, "This url should be in a try"


sample question surveys ~ {'sample question surveys': [(7.0, 'http://www.surveyconnect.com'), (4.0, 'http://www.custominsight.com'), (10.0, 'http://www.askemployees.com'), (1.0, 'http://www.lg-employers.gov.uk'), (nan, nan)]}
sample questions immigration interview ~ {'sample questions for immigration interview': [(nan, nan)]}
sample questions interview ~ {'sample questions for interview': [(1.0, 'http://www.quintcareers.com')]}
sample questions family interview ~ {'sample questions for family interview': [(2.0, 'http://www.grandparents-day.com'), (5.0, 'http://www.quintcareers.com'), (3.0, 'http://jobsearchtech.about.com')]}
sample questions sociology race ethnicity ~ {'sample questions sociology race and ethnicity': [(nan, nan)]}
sample questions biology ~ {'sample questions biology': [(3.0, 'http://www.utexas.edu'), (6.0, 'http://www.troy.k12.ny.us')]}
sample questions us citizenship test ~ {'sample questions for us citizenship test': [(1.0, 'http://uscis.gov')]}
sample questionarie 

## 2. [15] Write a suggest function which is non-sensitive to stop words

Suggest options for user query based on Trie you just built.
Output results sorted by frequency, print query count for each suggestion. If there is an url available, print the url too. If multiple url-s are available, print the one with the highest rank (the less the better).

Pass the asserts.

Question for analysis: What is the empirical threshold for minimal prefix for suggest?

In [58]:
stops = set('a on at of to is from for and with using the in &'.split())

In [93]:
from typing import List, Tuple


def process_user_query(query:str, trie:pygtrie.CharTrie) -> List[Tuple[str, int, str]]:
    key = remove_stops(query.lower())
    try:
        results = []
        for key, queries in trie.iteritems(key):
            for query, ranks_urls in queries.items():
                if ranks_urls:
                    url = max(ranks_urls, key=lambda x: x[0])[1]
                else:
                    url = None
                results.append((query, len(ranks_urls), url))
        
        results = sorted(results, key=lambda x: -x[1])
        return results
    except:
        return []

In [96]:
from typing import List

def complete_user_query(query: str, trie:pygtrie.CharTrie, top_k=5) -> List[str]:

    results = []
    processed_query = process_user_query(query, trie)
    for result in processed_query:
        if len(results) == top_k:
            break
        if result[0] not in results:
            results.append(result[0])
    return results
    
inp = "trie"
print("Query:", inp)
print("Results:")
res = complete_user_query(inp, aol_trie)
print(res)

#NB we assume you return suggested query string only
assert res[0] == "tried and true tattoo"
assert res[1] == "triest" or res[1] == "triethanalomine"

assert "boys and girls club of conyers georgia" \
            in complete_user_query("boys girls club conyers", aol_trie, 10), "Should be here"

Query: trie
Results:
['tried and true tattoo', 'triest', 'triethanalomine', 'tried and failed', 'when you tried and failed']


## 3. Measure suggest speed ##

### 3.1. [10] Full Trie test

Check how fast your search is working. Consider changing your code if it takes too long on average.

Sucess criterion:
- there is an average and a standard deviation for **multiple runs** of the given bucket.
- there is an average and a standard deviation for **multiple runs** of naive search in the unindexed dataset.

In [63]:
import time
def measure_exectime_in_ms(func, *args, **kwargs) -> float:
    start_time = time.time()
    func(*args, **kwargs)
    end_time = time.time()
    return (end_time-start_time) * 1000

In [64]:
def average(miliseconds:list)->float:
    return sum(miliseconds) / len(miliseconds)

In [67]:
def naive_search(data: pd.DataFrame, query: str):
    results = []
    for row_query in data["Query"].tolist():
        if row_query.startswith(query):
            results.append(row_query)
    
    return results

In [68]:
import statistics
inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell",
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]

n_runs = 5

for query in inp_queries:
    excetime = [measure_exectime_in_ms(complete_user_query, query, aol_trie) for _ in range(n_runs)]
    print(f"Trie - average execution time: {format(average(excetime), '.4f')} ms - {query=}")
    print(f"Trie - standev execution time: {format(statistics.stdev(excetime), '.4f')} ms - {query=}")

print("\n")

for query in inp_queries:
    excetime = [measure_exectime_in_ms(naive_search, aol_data, query) for _ in range(n_runs)]
    print(f"Naive search - average execution time: {format(average(excetime), '.4f')} ms - {query=}")
    print(f"Naive search - standev execution time: {format(statistics.stdev(excetime), '.4f')} ms - {query=}")


#TODO: measure average execution time and standard deviation (in milliseconds) per query and print it out
# Repeat this for index and for no index.

Trie - average execution time: 17.5582 ms - query='inf'
Trie - standev execution time: 0.9066 ms - query='inf'
Trie - average execution time: 29.8549 ms - query='the best '
Trie - standev execution time: 3.8670 ms - query='the best '
Trie - average execution time: 0.0000 ms - query='information retrieval'
Trie - standev execution time: 0.0000 ms - query='information retrieval'
Trie - average execution time: 0.0000 ms - query='sherlock hol'
Trie - standev execution time: 0.0000 ms - query='sherlock hol'
Trie - average execution time: 0.0000 ms - query='carnegie mell'
Trie - standev execution time: 0.0000 ms - query='carnegie mell'
Trie - average execution time: 0.3507 ms - query='babies r'
Trie - standev execution time: 0.4803 ms - query='babies r'
Trie - average execution time: 15.9065 ms - query='new york'
Trie - standev execution time: 0.1981 ms - query='new york'
Trie - average execution time: 0.0000 ms - query='googol'
Trie - standev execution time: 0.0000 ms - query='googol'
Trie 

## 4. [10] Add spellchecking to your suggest

Try to make your search results as close as possible. Compare top-5 results of each query with top-5 results for corrected.

You can use use [pyspellchecker](https://pypi.org/project/pyspellchecker/) `candidates()` call, or use any other spellchecker implementation.

In [ ]:
!pip install pyspellchecker

In [106]:
from spellchecker import SpellChecker
from itertools import product

def complete_user_query_with_spellchecker(query:str, trie:pygtrie.CharTrie, top_k=5) -> List[str]:
    checker = SpellChecker(language='en')
    query = query.lower()
    tokens = query.split()
    words = [token for token in tokens if token.isalnum()]
    wrong_words = list(checker.unknown(words))
    wrong_words_candidates = []
    for word in wrong_words:
        candidates = list(checker.candidates(word)) + [word]
        wrong_words_candidates.append(candidates)

    wrong_words_candidates = list(product(*wrong_words_candidates))
    suggestions = []
    for candidates in wrong_words_candidates:
        fixed = tokens
        for wrong, fix in zip(wrong_words, candidates):
            if wrong in fixed:
                fixed = [fix if token == wrong else token for token in fixed]
        suggestions.append(' '.join(fixed))


    suggestions_results = [process_user_query(suggestion, trie) for suggestion in suggestions]

    results = [result for suggestion_results in suggestions_results for result in suggestion_results]

    results = sorted(results, key=lambda x: -x[1])

    unique_results = []
    for result in results:
        if len(unique_results) == top_k:
            break
        
        if result[0] not in unique_results:
            unique_results.append(result[0])
    return unique_results


In [108]:
inp_queries = ["inormation retrieval", "shelrock hol", "carnagie mell", "babis r", "Barrbara "]
inp_queries_corrected = ["information retrieval", "sherlock hol", "carnegie mell", "babies r", "Barbara "]

for q, qc in zip(inp_queries, inp_queries_corrected):
    res1 = complete_user_query(qc, aol_trie, 5)
    res2 = complete_user_query_with_spellchecker(q, aol_trie, 5)
    assert  res1 == res2, "Assert {} and {} give different results".format(q, qc)
